First you'll need to select which hardware setup you have. You'll need to select both a SCOPETYPE and a PLATFORM. SCOPETYPE can either be 'OPENADC' for the CWLite/CW1200 or 'CWNANO' for the CWNano. PLATFORM is the target device, with 'CWLITEARM'/'CW308_STM32F3' being the best supported option, followed by 'CWLITEXMEGA'/'CW308_XMEGA', then by 'CWNANO'. As of CW 5.4, you can select the SimpleSerial version used. For example:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_2_1'

This code will connect the scope and do some basic setup. We're now just going to use a special setup script to do this. This script contains the commands we ran seperately before.

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

The following code will build the firmware for the target.

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/basic-passwdcheck
make PLATFORM=CWNANO CRYPTO_TARGET=NONE SS_VER=SS_VER_2_1 -j V=1

This script uses the ChipWhisperer API to connect to the vault over the serial link, read the boot message, send an incorrect password, and capture the response — the goal is to obtain a “fail” reference trace useful for locating the verification routine, synchronizing the scope trigger, and serving as a baseline for SPA/CPA analysis.

In [ ]:
import chipwhisperer as cw
 
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
target.baud = 9600

# Use the raw serial object, not SimpleSerial parsing
print("Boot message:", target.ser.read(50))   # read up to 50 bytes
 
# Send a wrong password
target.ser.write(b"1234\n")
print("Response:", target.ser.read(50))
scope.dis()